In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="Paired")

In [ ]:
def load_prices(path):
    df = pd.read_csv(path)
    df['Data'] = pd.to_datetime(df['Data'])
    df.set_index('Data').sort_index()
    return df['Zamkniecie']

def daily_returns(prices):
    '''Liczy dzienne zwroty proste.'''
    return prices.pct_change().dropna(how='all')

def average_return(daily_returns, trading_days):
    '''Liczy średni zwrot w skali roku.'''
    return (1 + daily_returns.mean()) ** trading_days - 1

def cumulative_return(daily_returns):
    '''Liczy skumulowaną stopę zwrotu dla dziennych stóp zwrotu.'''
    return (1 + daily_returns).prod() - 1

def volatility(daily_returns, trading_days):
    '''Liczy roczną zmienność na podstawie dziennych zwrotów.'''
    return daily_returns.std(ddof=1) * np.sqrt(trading_days)

def sharpe_ratio(daily_returns, risk_free_rate, trading_days):
    '''Liczy roczny wskaźnik Sharpe'a dla dziennych prostych zwrotów.'''
    daily_risk_free_rate = (1 + risk_free_rate) ** (1 / trading_days) - 1
    return ((daily_returns.mean() - daily_risk_free_rate) / daily_returns.std(ddof=1)) * np.sqrt(trading_days)

In [31]:
risk_free_rate = 0.0595 # https://www.obligacjeskarbowe.pl/oferta-obligacji/obligacje-3-letnie-tos/tos0128/
trading_days = 252

In [32]:
spx = load_prices('https://raw.githubusercontent.com/kornelia-kozaczewska/portfolio_analysis/refs/heads/main/data/%5Espx_d.csv')
ibca = load_prices('https://raw.githubusercontent.com/kornelia-kozaczewska/portfolio_analysis/refs/heads/main/data/ibca_de_d.csv')
mwig40 = load_prices('https://raw.githubusercontent.com/kornelia-kozaczewska/portfolio_analysis/refs/heads/main/data/mwig40tr_d.csv')

usdpln = load_prices('https://raw.githubusercontent.com/kornelia-kozaczewska/portfolio_analysis/refs/heads/main/data/usdpln_d.csv')
eurpln = load_prices('https://raw.githubusercontent.com/kornelia-kozaczewska/portfolio_analysis/refs/heads/main/data/eurpln_d.csv')

In [33]:
usdpln = usdpln.reindex(spx.index, method="ffill")
eurpln = eurpln.reindex(ibca.index, method="ffill")

spx_pln = spx * usdpln
ibca_pln = ibca * eurpln

spx_pln = spx_pln.reindex(mwig40.index, method='ffill')
ibca_pln = ibca_pln.reindex(mwig40.index, method='ffill')

### Wariant 60/40:

In [34]:
portfolio_60_40_PLN = ibca_pln * .6 + mwig40 * .4
portfolio_60_40_PLN_US = ibca_pln * .6 + mwig40 * .2 + spx_pln * .2

In [43]:
daily_60_40_PLN = daily_returns(portfolio_60_40_PLN)

avg_60_40_PLN = average_return(daily_60_40_PLN, trading_days)
cumulative_60_40_PLN = cumulative_return(daily_60_40_PLN)
sharpe_60_40_PLN = sharpe_ratio(daily_60_40_PLN, risk_free_rate, trading_days)

print('-----60/40 PLN-----')
print(f'Średni roczny zwrot: {(avg_60_40_PLN * 100).round(3)}%')
print(f'Skumulowany zwrot: {(cumulative_60_40_PLN * 100).round(3)}%')
print(f'Sharpe roczny: {sharpe_60_40_PLN.round(3)}.')

-----60/40 PLN-----
Średni roczny zwrot: 26.643%
Skumulowany zwrot: 56.027%
Sharpe roczny: 1.197.


In [42]:
daily_60_40_PLN_US = daily_returns(portfolio_60_40_PLN_US)

avg_60_40_PLN_US = average_return(daily_60_40_PLN_US, trading_days)
cumulative_60_40_PLN_US = cumulative_return(daily_60_40_PLN_US)
sharpe_60_40_PLN_US = sharpe_ratio(daily_60_40_PLN_US, risk_free_rate, trading_days)

print('-----60/40 PLN + US-----')
print(f'Średni roczny zwrot: {(avg_60_40_PLN_US * 100).round(3)}%')
print(f'Skumulowany zwrot: {(cumulative_60_40_PLN_US * 100).round(3)}%')
print(f'Sharpe roczny: {sharpe_60_40_PLN_US.round(3)}.')

-----60/40 PLN + US-----
Średni roczny zwrot: 19.555%
Skumulowany zwrot: 39.978%
Sharpe roczny: 0.929.
